In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 8: Time Series and Survival analysis (Lectures 19 and 20) 

**Due date: see the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points:4}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 4. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture, we had different measurements for each Location. 

We want you to consider this for the avocado prices dataset. For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

It seems like we have separate measurements for both categorical features in this data set: "region" and "type". When we sort the dataframe by date, we see that, for the same date, there are measurements for different regions. When we further sort the dataframe by region (as well as date), we see that, for the same date AND region, there are separate measurements for the different types.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          18249 non-null  datetime64[ns]
 1   AveragePrice  18249 non-null  float64       
 2   Total Volume  18249 non-null  float64       
 3   4046          18249 non-null  float64       
 4   4225          18249 non-null  float64       
 5   4770          18249 non-null  float64       
 6   Total Bags    18249 non-null  float64       
 7   Small Bags    18249 non-null  float64       
 8   Large Bags    18249 non-null  float64       
 9   XLarge Bags   18249 non-null  float64       
 10  type          18249 non-null  object        
 11  year          18249 non-null  int64         
 12  region        18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


It seems like type and region are the only categorical variables in this dataset. Looking at their value counts, we see two potential values for the type feature and many different values (~50) for the region feature.

In [10]:
df["type"].value_counts()

type
conventional    9126
organic         9123
Name: count, dtype: int64

In [11]:
df["region"].value_counts()

region
Albany                 338
Sacramento             338
Northeast              338
NorthernNewEngland     338
Orlando                338
Philadelphia           338
PhoenixTucson          338
Pittsburgh             338
Plains                 338
Portland               338
RaleighGreensboro      338
RichmondNorfolk        338
Roanoke                338
SanDiego               338
Atlanta                338
SanFrancisco           338
Seattle                338
SouthCarolina          338
SouthCentral           338
Southeast              338
Spokane                338
StLouis                338
Syracuse               338
Tampa                  338
TotalUS                338
West                   338
NewYork                338
NewOrleansMobile       338
Nashville              338
Midsouth               338
BaltimoreWashington    338
Boise                  338
Boston                 338
BuffaloRochester       338
California             338
Charlotte              338
Chicago              

In [12]:
df_date = df.sort_values(by="Date", ascending=False)
df_date.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2018-03-25,2.02,13379.77,86.84,5923.16,98.37,7271.40,6881.80,389.60,0.00,organic,2018,RaleighGreensboro
0,2018-03-25,1.34,1774776.77,63905.98,908653.71,843.45,801373.63,774634.09,23833.93,2905.61,conventional,2018,NewYork
0,2018-03-25,1.45,121917.39,1929.39,18391.86,110.05,101486.09,85313.41,16172.68,0.00,organic,2018,Southeast
0,2018-03-25,1.19,450658.34,205952.19,73267.19,4313.11,167125.85,132488.40,31527.74,3109.71,conventional,2018,SouthCarolina
0,2018-03-25,1.40,524265.69,103573.88,149867.07,998.53,269826.21,155866.80,113666.70,292.71,conventional,2018,Seattle


In [13]:
df_date = df.sort_values(by=["Date", "region"], ascending=False)
df_date.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2018-03-25,0.84,965185.06,438526.12,199585.90,11017.42,316055.62,153009.89,160999.10,2046.63,conventional,2018,WestTexNewMexico
0,2018-03-25,1.62,15303.40,2325.30,2171.66,0.00,10806.44,10569.80,236.64,0.00,organic,2018,WestTexNewMexico
0,2018-03-25,0.93,7667064.46,2567279.74,1912986.38,118289.91,3068508.43,1309580.19,1745630.06,13298.18,conventional,2018,West
0,2018-03-25,1.60,271723.08,26996.28,77861.39,117.56,166747.85,87108.00,79495.39,144.46,organic,2018,West
0,2018-03-25,1.03,43409835.75,14130799.10,12125711.42,758801.12,16394524.11,12540327.19,3544729.39,309467.53,conventional,2018,TotalUS


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

In this dataset, the measurements are generally spaced equally. They are separated by 7 days. The only exception is for the `region` WestTexNewMexico of `type` organic where the spacings are mostly 7 days but there is also spacing of 14 and 21 days. In terms of missing values, looking at the outputs from `size()`, the measurement differences, and `isnat()`, we can see that there is 1 missing measurement for each type at every region.

In [14]:
region_type = df.groupby(['region', 'type'])
print(region_type.size())

region               type        
Albany               conventional    169
                     organic         169
Atlanta              conventional    169
                     organic         169
BaltimoreWashington  conventional    169
                                    ... 
TotalUS              organic         169
West                 conventional    169
                     organic         169
WestTexNewMexico     conventional    169
                     organic         166
Length: 108, dtype: int64


In [15]:
# Adapted from Lecture 19
for (region, type), group in region_type:
    diff = group["Date"].sort_values().diff()
    print("(%s, %s): %s" % (region, type, diff.value_counts()))
    print('\n')

(Albany, conventional): Date
7 days    168
Name: count, dtype: int64


(Albany, organic): Date
7 days    168
Name: count, dtype: int64


(Atlanta, conventional): Date
7 days    168
Name: count, dtype: int64


(Atlanta, organic): Date
7 days    168
Name: count, dtype: int64


(BaltimoreWashington, conventional): Date
7 days    168
Name: count, dtype: int64


(BaltimoreWashington, organic): Date
7 days    168
Name: count, dtype: int64


(Boise, conventional): Date
7 days    168
Name: count, dtype: int64


(Boise, organic): Date
7 days    168
Name: count, dtype: int64


(Boston, conventional): Date
7 days    168
Name: count, dtype: int64


(Boston, organic): Date
7 days    168
Name: count, dtype: int64


(BuffaloRochester, conventional): Date
7 days    168
Name: count, dtype: int64


(BuffaloRochester, organic): Date
7 days    168
Name: count, dtype: int64


(California, conventional): Date
7 days    168
Name: count, dtype: int64


(California, organic): Date
7 days    168
Name: count, dt

In [16]:
# Adapted from https://numpy.org/doc/stable/reference/generated/numpy.isnat.html
# checking for missing values
for (region, type), group in region_type:
    diff = group["Date"].sort_values().diff()
    missing = np.isnat(diff).value_counts()[1]
    print("Number of missing values in (%s, %s): %s" % (region, type, missing)) 

Number of missing values in (Albany, conventional): 1
Number of missing values in (Albany, organic): 1
Number of missing values in (Atlanta, conventional): 1
Number of missing values in (Atlanta, organic): 1
Number of missing values in (BaltimoreWashington, conventional): 1
Number of missing values in (BaltimoreWashington, organic): 1
Number of missing values in (Boise, conventional): 1
Number of missing values in (Boise, organic): 1
Number of missing values in (Boston, conventional): 1
Number of missing values in (Boston, organic): 1
Number of missing values in (BuffaloRochester, conventional): 1
Number of missing values in (BuffaloRochester, organic): 1
Number of missing values in (California, conventional): 1
Number of missing values in (California, organic): 1
Number of missing values in (Charlotte, conventional): 1
Number of missing values in (Charlotte, organic): 1
Number of missing values in (Chicago, conventional): 1
Number of missing values in (Chicago, organic): 1
Number of m

/var/folders/p5/hwwszq5n2kv_18jljl_3nq1h0000gn/T/ipykernel_41171/1436522757.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  missing = np.isnat(diff).value_counts()[1]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

Looking at the unique values in `region` column, there are overlapping regions. For this feature, names of cities, states, regions, and the country itself are included and these overlap with each other. For example, both 'LosAngeles' and 'California' are included here when 'LosAngeles' is a city in 'California'. Similarly, 'SouthCentral' is a region used to collectively identify a few states in the South of the US, and 'Houston' is one of the cities within in this region, but in this dataset, both 'Houston' and 'SouthCentral' are included as individual values. 'TotalUS', from its name at least, refers to the entire country however it is included as an individual value along with other US cities, states, and regions.

In [17]:
df['region'].unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2023W1/tree/main/lectures), with some improvements.

In [18]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [19]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [20]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [21]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

### 1.4 `AveragePrice` baseline 
rubric={autograde:4}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

In [22]:
y_train_actual = df_train["AveragePriceNextWeek"]
y_train_baseline = df_train["AveragePrice"]

y_test_actual = df_test["AveragePriceNextWeek"]
y_test_baseline = df_test["AveragePrice"]

In [23]:
train_r2 = r2_score(y_train_actual, y_train_baseline)
train_r2

0.8285800937261841

In [24]:
test_r2 = r2_score(y_test_actual, y_test_baseline)
test_r2

0.7631780188583048

In [25]:
grader.check("q1.4")

q1.4 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 10

We have decided to experiment with 4 approaches: default model without encoding the date, model encoding date as a number (Days since split_date: September 25 2017), model encoding date as a categorical variable (One Hot Encoding by Month), and model using lag-based features for AveragePrice (Average Price last week, Average Price 2 weeks ago, Average Price 3 weeks ago)

The best approach was the model encoding date as a categorical variable (One Hot Encoding by Month) with a test score of 0.8. This model had the highest test score as compared to all 3 other models with a test score of 0.79 and 0.78. This indicates that this model is most likely to generalize the best on new unseen data (e.g. test data) as compared to the other models. This makes sense as ridge is a linear model that can only learn a linear function of the month of the year (encoded as categorical variable). The one-hot encoding of date feature transforms the cyclic temporal feature into a format where its impact on the target variable can be independently and linearly modelled, enabling the Ridge linear model to capture and use these cyclic patterns.

The worst approach was the model encoding date as a number (Days since split_date: September 25 2017) with a test score of 0.78, which is worse than the default model without encoding the date (test score = 0.79). This is expected as the Ridge linear model is not able to capture the periodic/cyclic (non-linear) pattern in the data given that the encoded date feature is numeric.

The model using lag-based features for AveragePrice performed similarly to the default model with a test score of 0.79 but had the highest train score of 0.86. Since the test score is not the highest, this model is not the best model. Furthermore, having the highest train score indicates that this model has the highest chance of overfitting on the train data, which further prevents it from being the best model.

In [26]:
# Adapted from Lecture 19
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import Ridge

def preprocess_features(
    train_df,
    test_df,
    numeric_features,
    categorical_features,
    drop_features,
    target
):

    all_features = set(numeric_features + categorical_features + drop_features + target)
    if set(train_df.columns) != all_features:
        print("Missing columns", set(train_df.columns) - all_features)
        print("Extra columns", all_features - set(train_df.columns))
        raise Exception("Columns do not match")

    numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
    categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        OneHotEncoder(handle_unknown="ignore", sparse_output=False),
    )

    preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )
    preprocessor.fit(train_df)
    ohe_feature_names = (
        preprocessor.named_transformers_["pipeline-2"]
        .named_steps["onehotencoder"]
        .get_feature_names_out(categorical_features)
        .tolist()
    )
    new_columns = numeric_features + ohe_feature_names

    X_train_enc = pd.DataFrame(
        preprocessor.transform(train_df), index=train_df.index, columns=new_columns
    )
    X_test_enc = pd.DataFrame(
        preprocessor.transform(test_df), index=test_df.index, columns=new_columns
    )

    y_train = train_df["AveragePriceNextWeek"]
    y_test = test_df["AveragePriceNextWeek"]

    return X_train_enc, y_train, X_test_enc, y_test, preprocessor

In [27]:
numeric_features = [
    "AveragePrice",
    "Total Volume",
    "4046",
    "4225",
    "4770",
    "Total Bags",
    "Small Bags",
    "Large Bags",
    "XLarge Bags",
    "year",
]
categorical_features = [
    "type",
    "region",
]
drop_features = ["Date"]
target = ["AveragePriceNextWeek"]

In [28]:
# Adapted from Lecture 19
X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features,
    categorical_features,
    drop_features, target
)

In [29]:
# Adapted from Lecture 19
def score_lr_print_coeff(preprocessor, train_df, y_train, test_df, y_test, X_train_enc):
    lr_pipe = make_pipeline(preprocessor, Ridge())
    lr_pipe.fit(train_df, y_train)
    print("Train score: {:.2f}".format(lr_pipe.score(train_df, y_train)))
    print("Test score: {:.2f}".format(lr_pipe.score(test_df, y_test)))
    lr_coef = pd.DataFrame(
        data=lr_pipe.named_steps["ridge"].coef_.flatten(),
        index=X_train_enc.columns,
        columns=["Coef"],
    )
    return lr_coef.sort_values(by="Coef", ascending=False)

In [30]:
# default model without encoding the date
score_lr_print_coeff(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.85
Test score: 0.79


,Coef
AveragePrice,0.328682
region_SanFrancisco,0.087368
region_HartfordSpringfield,0.084933
region_NewYork,0.067025
type_organic,0.049510
...,...
region_Denver,-0.045930
type_conventional,-0.049510
region_DallasFtWorth,-0.065811
region_SouthCentral,-0.065815


In [31]:
# Adapted from Lecture 19
# Model encoding date as a number (Days since split_date: September 25 2017)
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

first_day = df_train["Date"].min()

df_train = df_train.assign(
    Days_since=df_train["Date"].apply(lambda x: (x - first_day).days)
)
df_test = df_test.assign(
    Days_since=df_test["Date"].apply(lambda x: (x - first_day).days)
)

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features + ["Days_since"],
    categorical_features,
    drop_features,
    target
)

score_lr_print_coeff(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.85
Test score: 0.78


,Coef
AveragePrice,0.324498
region_SanFrancisco,0.091838
region_HartfordSpringfield,0.089592
region_NewYork,0.070944
region_Philadelphia,0.052258
...,...
region_Denver,-0.047782
type_conventional,-0.052177
region_DallasFtWorth,-0.069429
region_SouthCentral,-0.070072


In [32]:
# Adapted from Lecture 19
# Model encoding date as a categorical variable (One Hot Encoding by Month)

df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

df_train = df_train.assign(
    Month=df_train["Date"].apply(lambda x: x.month_name())
) 
df_test = df_test.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train, df_test, 
    numeric_features, 
    categorical_features + ["Month"], 
    drop_features,
    target
)

month_df = score_lr_print_coeff(preprocessor, df_train, y_train, df_test, y_test, X_train_enc)

Train score: 0.85
Test score: 0.80


In [33]:
# Adapted from Lecture 19
# Model using lag-based features of AveragePrice (Average Price last week, Average Price 2 weeks ago, Average Price 3 weeks ago)

df_hastarget_modified = create_lag_feature(df_hastarget, "AveragePrice", -1, ["region", "type"], "AveragePrice_lag1", clip=True)
df_hastarget_modified = create_lag_feature(df_hastarget_modified, "AveragePrice", -2, ["region", "type"], "AveragePrice_lag2", clip=True)
df_hastarget_modified = create_lag_feature(df_hastarget_modified, "AveragePrice", -3, ["region", "type"], "AveragePrice_lag3", clip=True)

df_train = df_hastarget_modified[df_hastarget_modified["Date"] <= split_date]
df_test  = df_hastarget_modified[df_hastarget_modified["Date"] >  split_date]

X_train_enc, y_train, X_test_enc, y_test, preprocessor = preprocess_features(
    df_train,
    df_test,
    numeric_features
    + ["AveragePrice_lag1", "AveragePrice_lag2", "AveragePrice_lag3"],
    categorical_features,
    drop_features,
    target
)

score_lr_print_coeff(
    preprocessor, df_train, y_train, df_test, y_test, X_train_enc
)

Train score: 0.86
Test score: 0.79


,Coef
AveragePrice,0.243340
AveragePrice_lag1,0.054982
region_SanFrancisco,0.050740
region_HartfordSpringfield,0.043354
AveragePrice_lag3,0.038736
...,...
region_Denver,-0.025660
type_conventional,-0.026386
region_DallasFtWorth,-0.035993
region_SouthCentral,-0.036761


<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Very short answer questions

Each question is worth 2 points.

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:4}

The following questions pertain to Lecture 19 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 4

1. An example of such a time series can be found in healthcare, particularly when monitoring patients with chronic conditions. This scenario can lead to time series data with unevenly spaced time points, since patient check-ups are often scheduled based on individual needs, health status changes, or the discretion of healthcare providers. Some patients may have more frequent appointments during critical periods, while others may follow a less frequent schedule during stable periods. This personalized approach to patient care results in a time series dataset where observations are not uniformly distributed.

2. The approach of creating lagged versions of features would likely struggle more with unequally spaced time points. Lagged features involve creating time-shifted versions of existing features, aligning them with previous time points. However, when time points are unevenly spaced and arbitrary, the fixed lag approach may not capture the temporal dynamics accurately. The gaps between observations could lead to situations where the lagged features do not align appropriately with the underlying temporal patterns, potentially resulting in misleading or incomplete representations of the time series data.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis
rubric={points:6}

The following questions pertain to Lecture 20 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

1. One of the main problems with labeling customers as "churned" or "not churned" and carrying out a standard supervised learning technique, such as binary classification, is that this kind of data is often collected at a certain point in time, and thus we would be predicting whether a customer would churn at that particular point. Therefore, any analysis that we perform will be based on data that is not exactly accurate and that likely includes major class imbalance, which in turn leads to bias in the model and a poor prediction of whatever class is the minority.

2. I believe the customer that is most likely to leave the service first is customer B. There are a few reasons for that: firstly, when you just join a service, you often have a month of free trial and you are still learning whether you'd like to keep the service or not. Generally, the longer one has been with a service, the more likely they are to churn based on probability. So customer B, who despite having been loyal to the service for a year, is statistically more likely to leave than customer A that just joined.

3. Considering that the survival function represents the probability that an event (such as customer churn) has not occurred by a given time, we can say that if the function is almost flat during a certain period, it suggests that the probability of the event occurring remains relatively constant or low during that time interval. Furthermore, it can also indicate how stable the behaviour of that customer is - if the curve is flat, it suggests that customers are not showing a significant propensity to churn, and the factors influencing churn are relatively constant or well-managed. This stability can arise from various scenarios, from customer loyalty and genuine enjoyment of the service to long contracts that force a customer to remain with the service.

<!-- END QUESTION -->

<br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)